In [ ]:
from os import listdir
from datetime import datetime
from typing import List

from google.colab import drive
from pandas import DataFrame, read_csv

In [ ]:
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
class Logger:
    def log_informacao(self, mensagem: str) -> None:
        """
        Padronização de print
        """
        print(f"{datetime.utcnow()} - {mensagem}")

In [ ]:
logger = Logger()


def criar_lista_de_arquivos() -> List[str]:
    """
    Gera a listagem de arquivos bronze
    """
    logger.log_informacao(
        mensagem=f"Criando lista de arquivos a serem processados")
    lista_arquivos_a_serem_processados = []
    lista_arquivos_bronze = listdir(
        path=(
            "/content/drive/MyDrive/engenharia_de_dados"
            "/a3/dados/bronze/ibge/"
        ))
    for arquivo in lista_arquivos_bronze:
        lista_arquivos_a_serem_processados.append(arquivo.replace(".csv", ""))
    return lista_arquivos_a_serem_processados


def executar_etl(arquivos_de_entrada: List[str]) -> None:
    """
    executa o processo de etl para cada tabela na listagem de entrada
    """
    for arquivo in arquivos_de_entrada:
        logger.log_informacao(mensagem=f"-------- Processando tabela {arquivo}")
        tabela_paic = extrair_tabela_paic_bronze(nome_da_tabela=arquivo)
        tabela_paic = transformar_tabela_paic(tabela_paic=tabela_paic)
        carregar_tabela_paic(tabela_paic=tabela_paic, nome_da_tabela=arquivo)


def extrair_tabela_paic_bronze(nome_da_tabela: str) -> DataFrame:
    """
    Lê a tabela localizada na camada bronze do drive
    """
    logger.log_informacao(mensagem=f"Lendo a tabela bronze {nome_da_tabela}")
    caminho = (
        "/content/drive/MyDrive/engenharia_de_dados"
        + f"/a3/dados/bronze/ibge/{nome_da_tabela}.csv"
    )
    tabela_paic = read_csv(filepath_or_buffer=caminho, sep=";")
    return tabela_paic


def transformar_tabela_paic(tabela_paic: DataFrame) -> DataFrame:
    """
    Performa as transformações necessárias para enriquecer os dados
    vindos da camada bronze
    """
    logger.log_informacao(mensagem="Iniciando transformações")
    tabela_paic = remover_linhas_de_categoria_da_tabela_paic(
        tabela_paic=tabela_paic)
    tabela_paic = criar_coluna_de_codigo_cnae(tabela_paic=tabela_paic)
    tabela_paic = criar_coluna_de_categoria_empresa(tabela_paic=tabela_paic)
    tabela_paic = mudar_tipo_das_colunas_para_object(tabela_paic=tabela_paic)
    tabela_paic = remover_espacamento_das_colunas(tabela_paic=tabela_paic)
    tabela_paic = de_para_coluna_valor(tabela_paic=tabela_paic)
    return tabela_paic


def remover_linhas_de_categoria_da_tabela_paic(
        tabela_paic: DataFrame) -> DataFrame:
    """
    Remove a totalização referente a totalização de cada divisão
    """
    logger.log_informacao(mensagem="Removendo totalização de divisao")
    categoria = [
        "Total das empresas",
        "Empresas entre 1 e 4 de PO - total",
        "Empresas entre 5 e 29 de PO - total",
        "Empresas com 30 ou mais de PO - total"]
    return (
        tabela_paic
        .query(expr="pessoal_ocupado_grupos_classes not in @categoria"))


def criar_coluna_de_codigo_cnae(tabela_paic: DataFrame) -> DataFrame:
    """
    Extrair o dado do código cnae da coluna pessoal_ocupado_grupos_classes
    em uma nova
    """
    logger.log_informacao(mensagem="Criando coluna codigo_cnae")
    tabela_paic.loc[:, ("codigo_cnae")] = (
        tabela_paic
        .loc[:, ("pessoal_ocupado_grupos_classes")]
        .str
        .extract(pat=r"(.+?(?= Empresas))"))
    return tabela_paic


def criar_coluna_de_categoria_empresa(tabela_paic: DataFrame) -> DataFrame:
    """
    Extrair o dado da categoria da empresa da
    coluna pessoal_ocupado_grupos_classes em uma nova
    """
    logger.log_informacao(mensagem="Criando coluna categoria_empresa")
    tabela_paic.loc[:, ("categoria_empresa")] = (
        tabela_paic
        .loc[:, "pessoal_ocupado_grupos_classes"]
        .str
        .extract(pat=r"(Empresas.*?PO)"))
    return tabela_paic


def mudar_tipo_das_colunas_para_object(tabela_paic: DataFrame) -> DataFrame:
    """
    Muda o tipo de total as colunas para string
    """
    logger.log_informacao(
        mensagem="Mudando o tipo de total as colunas para string")
    relacao_de_tipo_por_coluna = {}
    colunas = tabela_paic.columns
    for coluna in colunas:
        relacao_de_tipo_por_coluna[coluna] = "str"
    return tabela_paic.astype(dtype=relacao_de_tipo_por_coluna)


def remover_espacamento_das_colunas(tabela_paic: DataFrame) -> DataFrame:
    """
    Remove espaçamentos a esquerda e a direita
    """
    logger.log_informacao(mensagem="Removendo espacamento das colunas")
    colunas = tabela_paic.columns
    for coluna in colunas:
        if tabela_paic[coluna].dtype == "str":
            tabela_paic.loc[:, (coluna)] = (
                tabela_paic
                .loc[:, coluna]
                .str
                .strip())
    return tabela_paic


def de_para_coluna_valor(tabela_paic: DataFrame) -> DataFrame:
    """
    Faz o de para de valores invalidos de acordo com a legenda da pesquisa
    """
    logger.log_informacao(mensagem="Executando de para coluna valor")
    valor_de_para = {
        "-": "0",
        "..": "0",
        "...": "0",
        "X": "0"
    }
    tabela_paic.loc[:, ("valor")] = (
        tabela_paic.loc[:, ("valor")]
        .replace(to_replace=valor_de_para))
    return tabela_paic


def carregar_tabela_paic(tabela_paic: DataFrame, nome_da_tabela: str) -> None:
    """
    Carrega a tabela paic na pasta silver do drive
    """
    logger.log_informacao(mensagem=f"Carregando tabela {nome_da_tabela}")
    caminho = (
        "/content/drive/MyDrive/engenharia_de_dados"
        + f"/a3/dados/silver/ibge/{nome_da_tabela}_silver.csv"
    )
    colunas = [
        "ano_pesquisa", "variavel", "codigo_cnae", "categoria_empresa",
        "valor", "unidade"
    ]
    tabela_paic.to_csv(
        path_or_buf=caminho,
        sep=";",
        mode="w",
        index=False,
        columns=colunas,
        encoding="utf-8")

In [ ]:
arquivos_para_processar = criar_lista_de_arquivos()
executar_etl(arquivos_de_entrada=arquivos_para_processar)

2022-09-24 00:56:14.116648 - Criando lista de arquivos a serem processados
2022-09-24 00:56:14.118793 - -------- Processando tabela emprego_e_salario
2022-09-24 00:56:14.118853 - Lendo a tabela bronze emprego_e_salario
2022-09-24 00:56:14.129407 - Iniciando transformações
2022-09-24 00:56:14.131450 - Removendo totalização de divisao
2022-09-24 00:56:14.135363 - Criando coluna codigo_cnae
2022-09-24 00:56:14.139144 - Criando coluna categoria_empresa
2022-09-24 00:56:14.142206 - Mudando o tipo de total as colunas para string
2022-09-24 00:56:14.146398 - Removendo espacamento das colunas
2022-09-24 00:56:14.148607 - Executando de para coluna valor
2022-09-24 00:56:14.151121 - Carregando tabela emprego_e_salario
2022-09-24 00:56:14.163471 - -------- Processando tabela gasto_de_pessoal
2022-09-24 00:56:14.163622 - Lendo a tabela bronze gasto_de_pessoal
2022-09-24 00:56:14.171685 - Iniciando transformações
2022-09-24 00:56:14.171776 - Removendo totalização de divisao
2022-09-24 00:56:14.1748